<center><img src="../images/logo.png" alt="Your Image" style="width: 500px; height: auto"></center>

# Introduction to Echo State Networks based on the Example of Stock Price Prediction

In this notebook, we show how echo state networks (ESN) can be used for time series prediction and especially the prediction of stock price dynamics. In the following, the basic model is introduced with a focus on the design and hyperparameterization of echo state networks in python. Here, for the sake of simplicity, we limit ourselves to the case of a single stock prices time series.

## Import 

In [ ]:
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from scipy import linalg
from sklearn.linear_model import Ridge

## Load and plot data 

We collected stock data for Apple on 880 trading days, from January 1, 2020, to July 1, 2023. These stock data includes volumes and opening, closing,
highest, lowest, and adjusted closing prices. We load data using the Python package Yahoo Finance (yfinance).


In [ ]:
stock_data = yf.download("AAPL", start="2020-01-01", end="2023-07-15")
stock_data.head()

Next, we plot the collected closing prices, which are the volume-weighted average prices of all trades (including the last trade) within the last
minute of trading and are important for stock price forecasting.

In [ ]:
close_prices = stock_data['Close']

plt.figure(figsize=[15,5])
plt.plot(close_prices)
plt.title('Daily closing prices of chosen stock')
plt.ylabel('Closing price')
plt.xlabel('Day')

## Split data into training and test set


The data set should be split in a training and a test set, because the quality of the method is not based on the error produced with the data used for training, but it is based on the potential for generalisation of the algorithm, that is, the performance on data not previously seen. In an extreme case, overfitting can happen, such that the results for the given data are very good, but new data does not achieve an acceptable outcome. To measure this, a part of the data set should not be used for training the data, but for testing the algorithm after the training procedure.

The task here is to predict 2 days ahead by using the previous 800 days (training data) and do that for 80 future points (test data). So, in the end you will have a 80 time step prediction with dif = 2.

In [ ]:
train_len = 800
test_len = 80
dif = 2  # prediction horizon >=1

In [ ]:
data = np.array(close_prices)
X = data[0:train_len] 
y = data[dif:train_len+dif]
Xtest = data[train_len:train_len+test_len]
ytest = data[train_len+dif:train_len+test_len+dif]

## Building and training the Echo State Network

In this section, we build and train the echo state network (based on https://www.ai.rug.nl/minds/uploads/PracticalESN.pdf and https://github.com/alexander-rakhlin/ESN). 


Echo State Networks are a simple type of recurrent neural networks (RNN) which utilize reservoir computation.

The following section from [geeksforgeeks.org](https://www.geeksforgeeks.org/echo-state-network-an-overview/) states easy way to understand the basic concept of ESNs:

"Imagine an Echo State Network (ESN) in Python as a smart musical instrument player. Let’s say you’re trying to teach the player to mimic a song. The player has a large collection of notes (reservoir), and when you play the first few notes of the song (input), the player responds with its interpretation of the melody.
Now, here’s the trick: the player’s ability to interpret the song is based on its own unique style (randomly initialized reservoir), and it doesn’t change its playing style during practice (fixed weights). As you play more notes, the player adjusts how it combines those notes to match the song’s rhythm and melody. So, the player becomes really good at predicting the next note in the song, even if it hasn’t heard that specific sequence before."


The standard ESN structure includes input data (i.e., stock prices, $\mathbf{u}(n)$), reservoir activation states, and network output ($\mathbf{y}(n)$). The main difference between an ESN and a 'traditional' neural network is that the input and recurrent weight matrices $\mathbf{W}_{in}$ and $\mathbf{W}_{res}$ are initilized and set randomly when the network is established, such that little training is required. Solely, the output weight matrix $\mathbf{W}_{out}$ is trained. Moreover, the typical update equations are

$\tilde{x}(n) = \mathrm{tanh} (\mathbf{W}_{in} + \mathbf{W}_{res} \mathbf{x}(n-1))$, 

$\mathbf{x}(n) = (1-\alpha) \mathbf{x}(n-1) + \alpha \mathbf{\tilde{x}}(n)$ ,

where $x(n)$ is a vector of reservoir neuron activations, $\mathbf{\tilde{x}}(n)$ is its update at timestep $n$ and $\alpha$ describes the leaking rate, which can be regarded as the speed of the reservoir update dynamics.

In [ ]:
np.random.seed(42)
      
class ESN(object):

    def __init__(self, res_size=500, rho=0.9, cr=0.05, leaking_rate=0.2, W=None):

        self.res_size = res_size
        self.leaking_rate = leaking_rate
        self.rho = rho
        self.cr = cr

        if W is None:
            N = self.res_size * self.res_size
            W = np.random.rand(N) - 0.5
            zero_index = np.random.permutation(N)[int(N * self.cr * 1.0):]
            W[zero_index] = 0
            W = W.reshape((self.res_size, self.res_size))
            rhoW = max(abs(linalg.eig(W)[0]))
            W *= self.rho / rhoW
        else:
            assert W.shape[0] == W.shape[1] == self.res_size, "reservoir size mismatch"
        self.W = W

    def __init_states__(self, X, init_len, reset_state=True):

        self.S = np.zeros((len(X) - init_len, 1 + self.inSize + self.res_size))
        if reset_state:
            self.s = np.zeros(self.res_size)
        s = self.s.copy()

        for t, u in enumerate(X):
            s = (1 - self.leaking_rate) * s + self.leaking_rate *\
                                np.tanh(np.dot(self.Win, np.hstack((1, u))) +\
                                np.dot(self.W, s))
            if t >= init_len:
                self.S[t-init_len] = np.hstack((1, u, s))
        if reset_state:
            self.s = s

    def fit(self, X, y, lmbd=1e-6, init_len=100, init_states=True):
        assert len(X) == len(y), "input lengths mismatch."
        self.inSize =  1 if np.ndim(X) == 1 else X.shape[1]
        if init_states:
            self.Win = (np.random.rand(self.res_size, 1 + self.inSize) - 0.5) * 1
            self.__init_states__(X, init_len)
        self.ridge = Ridge(alpha=lmbd, fit_intercept=False,
                               solver='svd', tol=1e-6)
        self.ridge.fit(self.S, y[init_len:])
        return self

    def predict(self, X, init_states=True):
        if init_states:        
            self.__init_states__(X, 0, reset_state=False)
        y = self.ridge.predict(self.S)
        return y
        
  

We now establish the model with a reservoir size of 100 and set the hyperparameters to: 

- a spectral radius (rho) of 0.999
- a connectivity ratio (cr) of 0.02
- a leaking rate of 1
- a regularization coefficient (lmbd) of 1e-6

Then, the ESN is trained. Note, that the performance is significantly dependent on the set hyperparameters. General rules of thumb, which should be considered when setting hyperparameters in ESNs are published in "A Practical Guide to Applying Echo State Networks" by Lukosevicius (2012). Here, we concentrate on the following three key parameters: reservoir size, spectral radius, and the connectivity ratio of reservoirs. The reservoir size represents the number of neurons in the pool, is generally related to the number of samples and it has a certain effect on the network performance. A larger size means a more accurate stock forecast (however, keep in mind overfitting), while a too small size leads to underfitting. If the spectral radius of $W_{res}$ is less than 1, ESN will stay in an echo state (cf. Lukosevicius, 2012).  The connectivity ratio of a reservoir represents the connections between its neurons, meaning the more connections the stronger the nonlinear approximation ability (also cf. https://pdfs.semanticscholar.org/e3a7/2a2b7d6071461007112f12c8716529755d39.pdf).
 

In [ ]:
res_size = 500
rho = 0.999
cr = 0.02
leaking_rate = 1.
lmbd = 1e-6

esn = ESN(res_size=res_size, rho=rho, cr=cr, leaking_rate=leaking_rate)
    
esn.fit(X, y, init_len=100, lmbd=lmbd)
y_predicted = esn.predict(Xtest)

## Analysis of the model

We plot the training and test data to the predicted data of the ESN and evaluated the performance of ESN based on the root mean square error (RMSE) of forecast values ($y_t^p$) and actual values ($y_t$) at time $t$ with a total number of forecast values $N$:

$RMSE = \sqrt{\frac{1}{N} \sum_{t=1}^N ( y_t - y_t^p)^2}$

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(range(0,train_len+test_len),data[0:train_len+test_len],'b',alpha =0.5,label="Data")
plt.plot(range(train_len+dif,train_len+test_len+dif),y_predicted,'red', label="ESN")
plt.legend(loc='upper left',fontsize='x-large')
plt.title('Prediction window = 2')
plt.ylabel('Closing price')
plt.xlabel('Day')
lo,hi = plt.ylim()
plt.plot([train_len+dif,train_len+dif],[lo+np.spacing(1),hi-np.spacing(1)],'k:', linewidth=4)

In [ ]:
RMSE = np.sqrt(np.sum(np.square(ytest-y_predicted))/test_len)
print('RMSE:', RMSE)

The result shows, that the method works quite well, but keep in mind, that the above model was made with a prediction window of two days, meaning that we are only ever predicting 2 days into the future at any given time. 

Next, we analyse the dependency of the RMSE with regard to the prediction window length.

## RSME as function of prediction window length

In [ ]:
RMSE = []
wl = [1,2,3,4,5,6,7,8]

for i in wl:
    dif = i  
    data = np.array(close_prices)
    y = data[dif:train_len+dif]
    ytest = data[train_len+dif:train_len+test_len+dif]
    esn = ESN(res_size=res_size, rho=rho, cr=cr, leaking_rate=leaking_rate)
    esn.fit(X, y, init_len=100, lmbd=lmbd)
    y_predicted = esn.predict(Xtest)


    RMSE.append(np.sqrt(np.sum(np.square(ytest-y_predicted))/test_len))

plt.plot(wl,RMSE)
plt.xlabel('prediction window length')
plt.ylabel('RMSE')

In the future predictions, the error increases in time such that as the window length is increased the accuracy decreases. We can see this behavior in the plot above, where longer predictions show a larger RMSE.

## Changing the reservoir size

To get a feeling for the hyperparameters, the training may be conducted for various reservoir sizes.

In [ ]:
RMSE = []
res_size = [100,200,500,1000]

plt.figure(figsize=(15,5))
for i in res_size:
    dif = 2
    data = np.array(close_prices)
    esn = ESN(res_size=i, rho=rho, cr=cr, leaking_rate=leaking_rate)
    esn.fit(X, y, init_len=100, lmbd=lmbd)
    y_predicted = esn.predict(Xtest)

    plt.plot(range(train_len+dif,train_len+test_len+dif),y_predicted, label="ESN: res_size="+str(i))

    RMSE.append(np.sqrt(np.sum(np.square(ytest-y_predicted))/test_len))

plt.legend(loc='upper left',fontsize='x-large')
plt.title('Prediction window = 2')
plt.ylabel('Closing price')
plt.xlabel('Day')